# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
spark = SparkSession \
    .builder \
    .appName("AWS Spark Processes Ran Locally") \
    .getOrCreate()

In [3]:
# Creating dataframe for the demographics data
us_demographics_path = "us-cities-demographics.csv"
demographics = spark.read.csv(us_demographics_path, sep=";", header=True)

In [4]:
# Listing the schema for the demographics data
demographics.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [5]:
# Exploring data
demographics.select(['City', 'State', 'Race']).distinct().sort(['State', 'City']).show()

+----------+-------+--------------------+
|      City|  State|                Race|
+----------+-------+--------------------+
|Birmingham|Alabama|Black or African-...|
|Birmingham|Alabama|American Indian a...|
|Birmingham|Alabama|               Asian|
|Birmingham|Alabama|  Hispanic or Latino|
|Birmingham|Alabama|               White|
|    Dothan|Alabama|               Asian|
|    Dothan|Alabama|               White|
|    Dothan|Alabama|  Hispanic or Latino|
|    Dothan|Alabama|Black or African-...|
|    Dothan|Alabama|American Indian a...|
|    Hoover|Alabama|  Hispanic or Latino|
|    Hoover|Alabama|               White|
|    Hoover|Alabama|Black or African-...|
|    Hoover|Alabama|               Asian|
|Huntsville|Alabama|Black or African-...|
|Huntsville|Alabama|               Asian|
|Huntsville|Alabama|               White|
|Huntsville|Alabama|  Hispanic or Latino|
|Huntsville|Alabama|American Indian a...|
|    Mobile|Alabama|               White|
+----------+-------+--------------

In [6]:
# Exploring data for white people
# Using filter/where
# household size is the same for all races
demographics.select(['City','State','Average Household Size']).filter(demographics.Race == 'White').sort('Average Household Size', ascending=False).show()

+----------------+----------+----------------------+
|            City|     State|Average Household Size|
+----------------+----------+----------------------+
|       Brentwood|  New York|                  4.98|
|          Perris|California|                  4.78|
|       Santa Ana|California|                  4.58|
| Florence-Graham|California|                  4.57|
|         Lynwood|California|                  4.43|
|         Fontana|California|                  4.15|
|    Baldwin Park|California|                  4.13|
|          Oxnard|California|                  4.08|
|         Compton|California|                  4.08|
|      South Gate|California|                  3.97|
|       Poinciana|   Florida|                  3.93|
|   Moreno Valley|California|                   3.9|
|          Cicero|  Illinois|                  3.89|
|         Norwalk|California|                  3.88|
|   Jurupa Valley|California|                  3.87|
|          Rialto|California|                 

In [7]:
demographics.select(['City','State','Average Household Size']).sort('Average Household Size', ascending=False).show()

+---------------+----------+----------------------+
|           City|     State|Average Household Size|
+---------------+----------+----------------------+
|      Brentwood|  New York|                  4.98|
|      Brentwood|  New York|                  4.98|
|      Brentwood|  New York|                  4.98|
|      Brentwood|  New York|                  4.98|
|      Brentwood|  New York|                  4.98|
|         Perris|California|                  4.78|
|         Perris|California|                  4.78|
|         Perris|California|                  4.78|
|         Perris|California|                  4.78|
|      Santa Ana|California|                  4.58|
|      Santa Ana|California|                  4.58|
|      Santa Ana|California|                  4.58|
|      Santa Ana|California|                  4.58|
|      Santa Ana|California|                  4.58|
|Florence-Graham|California|                  4.57|
|Florence-Graham|California|                  4.57|
|Florence-Gr

In [8]:
demographics.select(['City','State', 'Male Population','Female Population', 'Count', 'Race' ,'Foreign-born','Average Household Size']).sort('Average Household Size', ascending=False).show()

+---------------+----------+---------------+-----------------+------+--------------------+------------+----------------------+
|           City|     State|Male Population|Female Population| Count|                Race|Foreign-born|Average Household Size|
+---------------+----------+---------------+-----------------+------+--------------------+------------+----------------------+
|      Brentwood|  New York|          31395|            32397| 46347|               White|       27058|                  4.98|
|      Brentwood|  New York|          31395|            32397|  4242|American Indian a...|       27058|                  4.98|
|      Brentwood|  New York|          31395|            32397|  8619|Black or African-...|       27058|                  4.98|
|      Brentwood|  New York|          31395|            32397| 40111|  Hispanic or Latino|       27058|                  4.98|
|      Brentwood|  New York|          31395|            32397|  1647|               Asian|       27058|        

In [9]:
demographics.select('Race').distinct().collect()

[Row(Race='Black or African-American'),
 Row(Race='Hispanic or Latino'),
 Row(Race='White'),
 Row(Race='Asian'),
 Row(Race='American Indian and Alaska Native')]

In [10]:
demographics.columns

['City',
 'State',
 'Median Age',
 'Male Population',
 'Female Population',
 'Total Population',
 'Number of Veterans',
 'Foreign-born',
 'Average Household Size',
 'State Code',
 'Race',
 'Count']

In [3]:
immigration_data_path = "immigration_data_sample.csv"
country_codes_path = "country_codes.csv"
immigrants = spark.read.csv(immigration_data_path, header=True)
countries = spark.read.csv(country_codes_path, header=True)

In [24]:
#immigrants.select('arrdate').show()

from pyspark.sql.functions import udf, col
from pyspark.sql.types import DateType
import pandas as pd

def convert_to_datetime(date):
    """
    Convert to yyyy-mm-dd format
    
    :return: date in yyyy-mm-dd format
    """   
    if date is not None:
        return pd.Timestamp('1960-1-1')+pd.to_timedelta(date, unit='D')
        
convert_to_datetime_udf = udf(convert_to_datetime, DateType())
fact_immigration = immigrants.withColumn('arrival_date', convert_to_datetime_udf(col('arrdate')))
fact_immigration.select(['arrdate', 'arrival_date']).show(10)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 603, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 449, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 251, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 71, in read_command
    command = serializer._read_with_length(file)
  File "/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.

In [12]:
# printing the schemas for immigrants table
# i94cit = Country Citizenship
# i94res = Country Residence

immigrants.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [18]:
# Counts of immigrants via different modes
immigrants.select([immigrants.i94mode]).groupBy(immigrants.i94mode).count().sort(immigrants.i94mode).show()

+-------+-----+
|i94mode|count|
+-------+-----+
|    1.0|  962|
|    2.0|   10|
|    3.0|   26|
|    9.0|    2|
+-------+-----+



In [13]:
# casting to a different type
immigrants.i94res.cast("int")
countries.country_code.cast("int")

Column<'CAST(country_code AS INT)'>

In [14]:
# What nationalities come to the US the most
immigrants.groupBy(immigrants.i94res).count() \
            .join(countries, immigrants.i94res.cast("int") == countries.country_code.cast("int"), 'inner') \
            .select('count',countries.country) \
            .sort('count', ascending=False) \
            .show()

+-----+--------------+
|count|       country|
+-----+--------------+
|  119|UNITED KINGDOM|
|   79|         JAPAN|
|   58|MEXICO Air Sea|
|   54|         CHINA|
|   52|       GERMANY|
|   51|     AUSTRALIA|
|   48|        FRANCE|
|   37|   SOUTH KOREA|
|   37|        BRAZIL|
|   34|         INDIA|
|   33|   NETHERLANDS|
|   22|    ARGENTINA |
|   19|      COLOMBIA|
|   19|        SWEDEN|
|   16|        TAIWAN|
|   16|        ISRAEL|
|   14|         ITALY|
|   14|   SWITZERLAND|
|   13|       IRELAND|
|   13|     VENEZUELA|
+-----+--------------+
only showing top 20 rows



In [15]:
# Counts on the types of immigrants
immigrants.select(immigrants.i94visa).groupBy(immigrants.i94visa).count().sort(immigrants.i94visa).show()

+-------+-----+
|i94visa|count|
+-------+-----+
|    1.0|  155|
|    2.0|  831|
|    3.0|   14|
+-------+-----+



In [16]:
# What are the most popular destinations for immigrants to visit
immigrants.select([immigrants.i94addr]).filter(immigrants.i94visa == 2).groupBy(immigrants.i94addr).count().sort('count', ascending=False).show()

+-------+-----+
|i94addr|count|
+-------+-----+
|     FL|  176|
|     NY|  148|
|     CA|  126|
|   null|   53|
|     HI|   51|
|     TX|   29|
|     GU|   27|
|     NV|   26|
|     IL|   24|
|     MA|   17|
|     NJ|   15|
|     GA|   14|
|     WA|   12|
|     NE|   10|
|     VA|   10|
|     DC|    9|
|     MD|    7|
|     LA|    7|
|     NC|    7|
|     CT|    6|
+-------+-----+
only showing top 20 rows



In [17]:
# What are the most popular destinations for immigrants to work at
immigrants.select([immigrants.i94addr]).filter(immigrants.i94visa == 1).groupBy(immigrants.i94addr).count().sort('count', ascending=False).show()

+-------+-----+
|i94addr|count|
+-------+-----+
|     CA|   33|
|     NY|   13|
|     TX|   11|
|     FL|   11|
|     NV|    8|
|     MA|    7|
|     IL|    7|
|     WA|    6|
|     GA|    5|
|   null|    5|
|     PA|    4|
|     NJ|    4|
|     MI|    4|
|     DC|    3|
|     AZ|    3|
|     MD|    3|
|     TN|    3|
|     SC|    2|
|     MN|    2|
|     VA|    2|
+-------+-----+
only showing top 20 rows



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.